In [183]:
import pandas as pd
from scipy import sparse
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

In [184]:
y_train = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_train.csv')

In [185]:
y_train.columns

Index(['Diabetes'], dtype='object')

In [186]:
y_test = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\y_test.csv')

In [187]:
x_train = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_train.csv')

In [188]:
x_test = pd.read_csv(r'C:\Projects_ciência_dados\DIABETES\data\processed\x_test.csv')

In [189]:
type(x_train)

pandas.core.frame.DataFrame

In [190]:
x_train.columns

Index(['onehot__Sex_Female', 'onehot__Sex_Male', 'onehot__Ethnicity_Asian',
       'onehot__Ethnicity_Black', 'onehot__Ethnicity_Hispanic',
       'onehot__Ethnicity_White', 'onehot__Physical_Activity_Level_High',
       'onehot__Physical_Activity_Level_Low',
       'onehot__Physical_Activity_Level_Moderate',
       'onehot__Alcohol_Consumption_Heavy',
       'onehot__Alcohol_Consumption_Moderate',
       'onehot__Smoking_Status_Current', 'onehot__Smoking_Status_Former',
       'onehot__Smoking_Status_Never', 'scaler__Age', 'scaler__BMI',
       'scaler__Waist_Circumference', 'scaler__Blood_Pressure_Systolic',
       'scaler__Blood_Pressure_Diastolic', 'scaler__Cholesterol_Total',
       'scaler__Cholesterol_HDL', 'scaler__Cholesterol_LDL', 'scaler__GGT',
       'scaler__Serum_Urate', 'scaler__Dietary_Intake_Calories',
       'scaler__Family_History_of_Diabetes',
       'scaler__Previous_Gestational_Diabetes'],
      dtype='object')

#### Grid trabalha com dados completos, não precisa separar em treino e teste, pois ele já faz isso com a validação cruzada

In [191]:
x_geral = pd.concat([x_train,x_test], axis=0)

In [192]:
y_geral = pd.concat([y_train,y_test], axis=0)

#### Gaussian

In [193]:
from sklearn.naive_bayes import BernoulliNB

In [194]:
bernoulli = BernoulliNB()
bernoulli.fit(x_train, y_train)
previsao_bernoulli = bernoulli.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [195]:
acuracia_bernoulli = accuracy_score(previsao_bernoulli, y_test)
print(f"Acurácia bernoulli: {acuracia_bernoulli}")

Acurácia bernoulli: 0.5576


In [196]:
print(classification_report(previsao_bernoulli,y_test))

              precision    recall  f1-score   support

           0       0.97      0.56      0.71      2428
           1       0.02      0.38      0.05        72

    accuracy                           0.56      2500
   macro avg       0.50      0.47      0.38      2500
weighted avg       0.94      0.56      0.69      2500



#### Árvores

In [197]:
parametros = {
    "max_depth":[500, 600, 650],
    "min_samples_split":[2,5,10],
    "min_samples_leaf":[1,5,10],
    "criterion":["gini", "entropy"]
}

In [198]:
from sklearn.tree import DecisionTreeClassifier

In [199]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#### Grid trabalha com dataset completo

In [200]:
randomized = RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42), param_distributions=parametros, verbose=True, n_jobs=4, cv=3, n_iter=20)
randomized.fit(x_train,y_train)
melhores_parametros = randomized.best_params_
melhor_resultado = randomized.best_score_


Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [201]:
print(melhor_resultado)

0.5212


In [202]:
print(melhores_parametros)

{'min_samples_split': 2, 'min_samples_leaf': 10, 'max_depth': 600, 'criterion': 'entropy'}


In [203]:
arbol = DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=1, max_depth=650, criterion='gini')
arbol.fit(x_train, y_train)
previsao_arbol = arbol.predict(x_test)

In [204]:
cross_arbol1 = cross_val_score(arbol,x_train,y_train, cv=5)
print(cross_arbol1.mean())

0.5029333333333333


In [205]:
print(accuracy_score(previsao_arbol,y_test))

0.5204


In [206]:
print(classification_report(previsao_arbol,y_test))

              precision    recall  f1-score   support

           0       0.56      0.58      0.57      1375
           1       0.47      0.45      0.46      1125

    accuracy                           0.52      2500
   macro avg       0.51      0.51      0.51      2500
weighted avg       0.52      0.52      0.52      2500



#### Profundidade da árvore

In [207]:
arbol.get_depth()

36

#### Numero de folhas

In [208]:
arbol.get_n_leaves()

np.int64(1630)

In [209]:
arbol.tree_

##### Serve para ver as features mais importantes, mas ele perdeu o nome das features

In [210]:
importances = pd.DataFrame({
    "features":x_train_DF.columns,
    "importancia":arbol.feature_importances_
}).sort_values(by="importancia", ascending=False)

NameError: name 'x_train_DF' is not defined

In [ ]:
importances.head()

#### Regressão logística

In [211]:
from sklearn.linear_model import LogisticRegression

In [212]:
logistic = LogisticRegression(max_iter=1000)
logistic.fit(x_train, y_train)
previsao_logistic = logistic.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [213]:
print(accuracy_score(previsao_logistic,y_test))

0.5636


In [214]:
print(classification_report(previsao_logistic,y_test))

              precision    recall  f1-score   support

           0       0.99      0.56      0.72      2483
           1       0.01      0.41      0.01        17

    accuracy                           0.56      2500
   macro avg       0.50      0.49      0.37      2500
weighted avg       0.99      0.56      0.72      2500



In [215]:
cross_logistic = cross_val_score(logistic, x_train, y_train, cv=5)
cross_logistic.mean()

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d

np.float64(0.5534666666666668)

#### Random

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#### Verificar se o random forest está overfitando ou underfitando, verificar os parâmetros default

In [ ]:
from scipy.stats import randint

In [ ]:
parametros = {
    "n_estimators": randint(100,301), # no randint coloca o maior e menor
    "min_samples_split": randint(10,30),
    "min_samples_leaf": randint(10,30),
    "max_features": ['log2','sqrt'],
    "max_depth": randint(10,20)
}

In [ ]:
randomized_random = RandomizedSearchCV(estimator=RandomForestClassifier(), param_distributions=parametros, n_jobs=4, verbose=True)
randomized_random.fit(x_train,y_train)
melhores_parametros = randomized_random.best_params_
melhor_resultado = randomized_random.best_score_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
print(melhor_resultado)

0.5573333333333333


In [ ]:
print(melhores_parametros)

{'max_depth': 16, 'max_features': 'log2', 'min_samples_leaf': 23, 'min_samples_split': 27, 'n_estimators': 149}


In [ ]:
random = RandomForestClassifier(verbose=True, n_jobs=4)
random.fit(x_train,y_train)
previsao_random = random.predict(x_test)
previsao_ingenua_random = random.predict(x_train)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
print(accuracy_score(previsao_random,y_test))

0.734


In [ ]:
print(accuracy_score(previsao_ingenua_random,y_train))

1.0


In [ ]:
cross_random = cross_val_score(random, x_train,y_train,cv=5)
print(f"Cross random: {cross_random.mean()}")

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[Parallel(

Cross random: 0.708


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
print(random.max_depth)

None


#### Número de árvores

In [ ]:
print(random.n_estimators)

100


#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(x_train,y_train)
previsao_KNN = KNN.predict(x_test)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
print(accuracy_score(previsao_KNN,y_test))

0.8292


In [ ]:
x_train_array = x_train.toarray()

In [ ]:
x_train_array

array([[0.        , 2.000001  , 2.30847842, ..., 4.32174931, 2.00019603,
        2.00108989],
       [0.        , 2.000001  , 2.30847842, ..., 5.16042265, 2.00019603,
        2.00108989],
       [0.        , 2.000001  , 0.        , ..., 2.7853444 , 0.        ,
        0.        ],
       ...,
       [0.        , 2.000001  , 2.30847842, ..., 4.5757236 , 0.        ,
        2.00108989],
       [2.000001  , 0.        , 2.30847842, ..., 5.48137917, 2.00019603,
        2.00108989],
       [2.000001  , 0.        , 2.30847842, ..., 3.72449109, 2.00019603,
        0.        ]], shape=(7500, 1880))

##### SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
parametros = {
    "C":[1,2,4,5,6],
    "kernel":['poly','rbf','sigmoid','linear']
}

In [ ]:
randomized_SVM = RandomizedSearchCV(estimator=SVC(), param_distributions=parametros, n_jobs=4, verbose=True)
randomized_SVM.fit(x_train,y_train)
melhores_parametros = randomized_SVM.best_params_
melhor_resultado = randomized_SVM.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print(melhor_resultado)

0.8552


In [ ]:
print(melhores_parametros)

{'kernel': 'sigmoid', 'C': 5}


In [ ]:
SVM = SVC()
SVM.fit(x_train,y_train)
previsao_SVM = SVM.predict(x_test)
previsao_ingenua = SVM.predict(x_train)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print(accuracy_score(previsao_ingenua,y_train))

0.9349333333333333


In [ ]:
cross_SVM = cross_val_score(SVM,x_train,y_train,cv=5)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d

In [ ]:
print(cross_SVM.mean())

0.8386666666666667


In [ ]:
print(accuracy_score(previsao_SVM,y_test))

0.8564


##### xgboost

In [ ]:
#### Ocorreu underfitting

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_params = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}

In [ ]:
randomized_xg = RandomizedSearchCV(estimator=XGBClassifier(), param_distributions=xgb_params, n_jobs=4, verbose=True)
randomized_xg.fit(x_train,y_train)
melhores_parametros = randomized_xg.best_params_
melhor_resultado = randomized_xg.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [ ]:
print(melhor_resultado)

0.7510666666666668


In [ ]:
print(melhores_parametros)

{'subsample': 1.0, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 9, 'learning_rate': 0.2, 'gamma': 0.2, 'colsample_bytree': 0.8}


In [ ]:
XG = XGBClassifier(n_estimator = 200, min_child_weight=1, max_depth=9, learning_rate=0.2, gamma=0.2, colsample_bytree = 0.8, subsample = 1.0)
XG.fit(x_train,y_train)
cross_xg = cross_val_score(XG,x_train,y_train,cv=5)
previsao_XG = XG.predict(x_test)
previsao_ingenua = XG.predict(x_train)

c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:33:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:33:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:33:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Projects_ciência_dados\DIABETES\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:33:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_esti

In [ ]:
print(accuracy_score(previsao_ingenua,y_train)) 

0.8632


In [ ]:
print(cross_xg.mean())

0.7458666666666667


In [ ]:
print(accuracy_score(previsao_XG,y_test))

0.766
